In [1]:
import numpy as np
# Import the Ising Model Functions Class
from functions import Ising_Functions

In [2]:
# All the parameters are set up for N = 100
kT = 0.2
N = 100
# Set up a zero array to contain random spin grid
spin = [[0 for col in range(N)] for row in range(N)]
HalfPos = int(N / 2) - 1
spin[HalfPos][HalfPos] = 1
spin[HalfPos + 1][HalfPos] = 1
spin[HalfPos][HalfPos + 1] = 1
spin[HalfPos + 1][HalfPos + 1] = 1
spin[0][0] = 1
spin[0][1] = 1
spin[1][0] = 1
spin[1][1] = 1

# Reformat the array so that it is numpy friendly
spin = np.array(spin)

In [3]:
spin_array_size = np.size(spin)
# Pick a random spin in the array "spin"
linearIndex = np.random.randint(1, spin_array_size)

In [4]:
get_values = Ising_Functions(spin, linearIndex, N).Fitness_Eqn_Val()
neighbours = get_values[0]
neighbours_sqrd = get_values[1]
one_minus_neighbour = get_values[2]
neighbour_spin_product = get_values[4]
print(neighbours, neighbours_sqrd, one_minus_neighbour, neighbour_spin_product)

0 0 4 0


In [ ]:
spin = 0
dE_0_1 = 0.1
dE_0_m1 = 0.1
# The probability of growing to 1
prob_g_1 = np.exp(-(dE_0_1 / kT))
# The probability of growing to -1
prob_g_m1 = np.exp(-(dE_0_m1 / kT))

# The fit_list is one which contains a boolean to determine if the fitness (energy) is negative,
# the probability of switching to that state if it is not favorable (due to detailed balance),
# and the final spin state.
fit_list = [
    {"bool": dE_0_1 < 0, "prob": prob_g_1, "spin": 1},
    {"bool": dE_0_m1 < 0, "prob": prob_g_m1, "spin": -1}]
allow_fit = sum([i["bool"] for i in fit_list])

# Determine which values are accepted
# Both are acceptable
if allow_fit > 1:
    prob_rand = np.random.rand()
    # Use partition function to find which transition occurs.
    prob_list = [j["prob"] for j in fit_list]
    G_1_or_m1 = Ising_Functions.partition_gillespie(prob_list, prob_rand)
    for k, l in enumerate(G_1_or_m1):
        if l:
            spin = fit_list[k]["spin"]
# Only one accepted.
elif allow_fit > 0:
    for j in fit_list:
        if j["bool"]:
            spin = j["spin"]
# Probability of transition still occurring due to detailed balance
else:
    prob_rand_g = np.random.rand()
    print(prob_rand_g, fit_list[0]["prob"], fit_list[1]["prob"])
    # The probability of both is selected
    # Use transition probabilities from detailed balance to find which transition occurs.
    allow_prob = sum((prob_rand_g < i["prob"] for i in fit_list))
    # Both probabilities from detailed balance are selected due to being above the random number selected.
    if allow_prob > 1:
        # Reroll number again to randomize which transition occurs.
        prob_rand_g = np.random.rand()
        # Use partition function to find which transition occurs.
        prob_list = [j["prob"] for j in fit_list]
        G_1_or_m1 = Ising_Functions.partition_gillespie(prob_list, prob_rand_g)
        for k, l in enumerate(G_1_or_m1):
            if l:
                spin = fit_list[k]["spin"]
    # Probability of 0 --> 1 selected or Probability of 0 --> -1 selected
    else:
        for j in fit_list:
            if prob_rand_g < j["prob"]:
                spin = j["spin"]
print(spin)